### 1 – Autenticação no Hugging Face Hub

Este trecho importa a função login do SDK Hugging Face Hub e abre uma sessão autenticada.
	•	new_session=False indica que, se já houver um token válido na máquina (variável HUGGINGFACE_HUB_TOKEN ou cache local), ele será reutilizado, evitando múltiplas autenticações interativas.
	•	A autenticação é necessária para baixar modelos privados ou com cotas restritas diretamente dos servidores da Hugging Face

In [ ]:
from huggingface_hub import login
login(new_session=False)


### 2 – Instalação das dependências

In [ ]:
# Dependencies installation
!pip install torch>=2.0.0 transformers>=4.30.0 huggingface_hub>=0.15.0 numpy>=1.21.0 accelerate>=0.20.0

<a href="https://colab.research.google.com/github/nelsonfrugeri-tech/playground-generative-ai/blob/master/logits_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 3 – Verificação de GPU (CUDA)
Este bloco confirma se o PyTorch reconhece uma GPU CUDA e exibe o nome do dispositivo 0.
	•	torch.cuda.is_available() retorna True quando há driver CUDA e GPU compatível.
	•	torch.cuda.get_device_name(0) mostra o modelo da placa (neste caso, A100).

Essas checagens evitam erros futuros—por exemplo, carregar o modelo em GPU quando a infraestrutura não oferece suporte.

In [ ]:
import torch
print(torch.cuda.is_available())  # Deve imprimir: True
print(torch.cuda.get_device_name(0))  # Deve mostrar: 'NVIDIA A100-SXM4-40GB'


### 4 – Seleção do Modelo

Define uma constante com o identificador do modelo no Hub. google/gemma-3-4b-pt é:
	•	Uma variante de 4 bilhões de parâmetros da família Gemma 3.
	•	Sufixo -pt indica que o checkpoint tem suporte (ou foco) para português.

Esse ID será reutilizado mais adiante para carregar AutoTokenizer e AutoModelForCausalLM, mantendo o código limpo e configurável.

In [ ]:
MODEL_ID = "google/gemma-3-4b-pt"

### 5 – Importações Principais

Este bloco reúne todas as dependências essenciais para geração de texto com controle de logits:
| **Componente**                                         | **Função no notebook**                                                                                                                     |
|--------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------- |
| `torch`                                                | Backend tensorial que executa operações em CPU ou GPU.                                                                                     |
| `pipeline`                                             | *Wrapper* de alto nível para inferência; simplifica o carregamento do modelo e do tokenizer.                                               |
| `LogitsProcessor`                                      | Classe-base para criar processadores que alteram logits antes do *sampling*.                                                               |
| `PreTrainedTokenizerBase`                              | Classe-base para todos os tokenizers pré-treinados; define a interface comum de **encode/decode** e o manuseio de vocabulário especial.    |
| `AutoTokenizer`, `AutoModelForCausalLM`                | Carregam automaticamente o tokenizer e o modelo correspondentes ao `MODEL_ID`.                                                             |



In [ ]:
import torch
from transformers import pipeline, LogitsProcessor, PreTrainedTokenizerBase, AutoTokenizer, AutoModelForCausalLM

### 5 - Classe `RulesLogitsProcessor`: Logits Masking Customizado para Tokens Proibidos

Esta classe personalizada de `LogitsProcessor` permite aplicar **Logits Masking** durante a geração de texto por modelos de linguagem (LLMs) usando Hugging Face Transformers.  
O objetivo é **bloquear a geração de palavras ou termos proibidos** (compliance, ética, branding etc) já no momento do sampling, evitando que o modelo produza saídas indesejadas.

- **`__init__`**: Recebe o tokenizer e uma lista de palavras proibidas, convertendo cada uma delas nos respectivos token_ids para aplicação eficiente do masking.
- **`apply_rules`**: Função utilitária que retorna `False` se qualquer palavra proibida já aparecer na sequência gerada até o momento.
- **`__call__`**: Método obrigatório do `LogitsProcessor`, chamado a cada etapa de geração.  
  - Zera a probabilidade (`-inf`) dos tokens proibidos, garantindo que não possam ser gerados.
  - Garante que o token especial EOS (fim de sequência) nunca seja bloqueado, evitando loops de geração.
  - Bloqueia explicitamente qualquer próximo token proibido na matriz de logits.

**Use Cases:**  
- Compliance regulatório
- Prevenção de respostas ofensivas
- Alinhamento a políticas internas de comunicação ou produto

In [ ]:
class RulesLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer, palavras_proibidas):
        self.tokenizer = tokenizer
        # Converte cada palavra proibida em lista de token_ids
        self.tokens_proibidos = set()
        for palavra in palavras_proibidas:
            for tid in tokenizer.encode(palavra, add_special_tokens=False):
                self.tokens_proibidos.add(tid)

    def apply_rules(self, seq):
        return not any(palavra in seq for palavra in self.palavras_proibidas)

    def __call__(self, input_ids: torch.LongTensor, input_logits: torch.FloatTensor) -> torch.FloatTensor:
        output_logits = input_logits.clone()
        eos_token_id = self.tokenizer.eos_token_id

        for idx, input_id in enumerate(input_ids):
            seq = self.tokenizer.decode(input_id)
            if not self.apply_rules(seq):
                # Permite sempre ao menos o EOS
                output_logits[idx] = -float("inf")
                output_logits[idx, eos_token_id] = 0

        # Também bloqueia qualquer próximo token proibido:
        output_logits[:, list(self.tokens_proibidos)] = -float("inf")
        return output_logits

### 6 - Carregamento do Modelo e Tokenizer

Neste trecho, o modelo de linguagem (LLM) e seu tokenizer correspondente são carregados a partir do Hugging Face Hub utilizando o identificador definido em `MODEL_ID`.  
- **`AutoTokenizer.from_pretrained(MODEL_ID)`**: Baixa e instancia o tokenizer apropriado para o modelo, responsável por converter texto em tokens e vice-versa.
- **`AutoModelForCausalLM.from_pretrained(MODEL_ID)`**: Baixa e instancia o modelo de linguagem pré-treinado, preparado para tarefas de geração textual (`causal language modeling`).

Esse passo é fundamental para configurar o pipeline de geração de texto com o modelo desejado, garantindo compatibilidade entre tokenizer e modelo.

In [ ]:
# Carrega modelo e tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

### 7 - Pipeline de Geração com Logits Masking e Palavras Proibidas

Neste trecho, é configurado o pipeline completo para geração de texto com bloqueio de termos indesejados:

- **Definição de palavras proibidas:**  
  Uma lista simples (`palavras_proibidas`) contém termos que devem ser evitados nas respostas geradas pelo modelo.

- **Pipeline Hugging Face:**  
  O pipeline é criado para tarefa de geração de texto (`text-generation`), conectando o modelo, tokenizer e configurando o uso de GPU se disponível.

- **Aplicação do Logits Masking:**  
  O `rules_processor` (`MyRulesLogitsProcessor`) recebe o tokenizer e as palavras proibidas, sendo passado ao pipeline via `logits_processor`, garantindo que as palavras bloqueadas nunca apareçam nas respostas.

- **Prompt estruturado:**  
  O `input_message` simula um cenário de atendimento ao cliente, com informações do sistema logístico e a pergunta do usuário.

- **Execução da geração:**  
  O pipeline gera a resposta considerando as restrições definidas, maximizando a chance de respostas adequadas, objetivas e dentro das regras especificadas.

Esse setup é ideal para cenários onde compliance e experiência do usuário exigem controle total sobre o que pode ou não ser gerado pelo assistente virtual.

In [ ]:
# Defina uma lista de palavras proibidas para a regra (exemplo simples)
palavras_proibidas = ["desculpe", "infelizmente"]

# Cria pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

rules_processor = MyRulesLogitsProcessor(tokenizer, palavras_proibidas)

# Defina o prompt de entrada
input_message = (
    "<|system|>\n"
    "Você é um atendente cordial, direto e focado na solução. "
    "Explique ao cliente o que será feito para resolver o problema. Seja transparente e objetivo. \n\n"
    "Abaixo segue informações do sistema de logística. \n\n"
    "Consulta ao sistema logístico: status do pedido #12345 — Pedido ATRASADO. Motivo: alto volume de entregas na região. Previsão de nova entrega: 2 dias úteis.\n\n"
    "<|user|>\n"
    "Meu pedido está atrasado e eu preciso de uma solução urgente. Não aceito mais desculpas, quero uma resposta clara e objetiva sobre o que será feito.\n"
    "<|assistant|>\n"
)


# Executa a geração
results = pipe(
    input_message,
    max_new_tokens=1024,
    do_sample=True,
    temperature=1.0,
    num_beams=5,
    logits_processor=[rules_processor],
    pad_token_id=tokenizer.eos_token_id
)

print("Resposta gerada:\n", results)

### 8 - Evolução na classe RulesLogitsProcessor - Estrutura Modular para Logits Masking — Interface, Implementação e Processor

Este bloco define uma arquitetura modular e extensível para aplicação de Logits Masking em modelos de linguagem, usando princípios de programação orientada a interfaces:

- **`MaskingRules` (Interface/ABC):**  
  Uma interface abstrata para a definição de regras de masking, exigindo a implementação do método `get_tokenid2prob`, que retorna um dicionário mapeando token IDs para suas probabilidades ajustadas.

- **`MaskingRulesImpl` (Implementação):**  
  Implementação concreta da interface `MaskingRules`.  
  - No construtor, converte cada termo e sua probabilidade em token IDs.  
  - Probabilidade 0 é automaticamente convertida para bloqueio total (`-inf`), outras probabilidades são aplicadas conforme desejado.

- **`RulesLogitsProcessor` (Custom LogitsProcessor):**  
  Classe personalizada que utiliza qualquer implementação de `MaskingRules` para aplicar logits masking durante a geração de texto.  
  - No método `__call__`, para cada token configurado, ajusta seu logit somando a probabilidade especificada, permitindo tanto bloqueio total quanto incentivo/desincentivo sutil.

**Benefícios deste design:**
- Separação clara de responsabilidades (regras vs. aplicação).
- Facilidade para testar, expandir ou trocar a lógica de regras.
- Pronto para cenários de compliance, branding, priorização de produtos e segurança textual.

In [ ]:
from typing import Dict
from abc import ABC, abstractmethod


class MaskingRules(ABC):
    @abstractmethod
    def get_tokenid2prob(self) -> Dict[int, float]:
      pass


class MaskingRulesImpl(MaskingRules):
    def __init__(self, tokenizer: PreTrainedTokenizerBase, regras_dict: Dict[str, float]):
        self.tokenid2prob: Dict[int, float] = {}
        for termo, prob in regras_dict.items():
            token_ids = tokenizer.encode(termo, add_special_tokens=False)
            for tid in token_ids:
                self.tokenid2prob[tid] = -float("inf") if prob == 0 else float(prob)

    def get_tokenid2prob(self) -> Dict[int, float]:
        return self.tokenid2prob

In [ ]:
class RulesLogitsProcessor(LogitsProcessor):
    def __init__(self, masking_rules: MaskingRules):
        self.tokenid2prob: Dict[int, float] = masking_rules.get_tokenid2prob()

    def __call__(
        self,
        input_ids: torch.LongTensor,
        scores: torch.FloatTensor
    ) -> torch.FloatTensor:
        output_logits: torch.FloatTensor = scores.clone()
        for tid, prob in self.tokenid2prob.items():
            output_logits[:, tid] += prob
        return output_logits

### 9 - Exemplo Prático: Priorização de Marcas em Recomendação de E-commerce com Logits Masking

Este bloco de código demonstra como usar o `RulesLogitsProcessor` para **influenciar a preferência do modelo** ao recomendar produtos (marcas de tênis esportivos) em um cenário de e-commerce:

- **Definição de regras de relevância:**  
  O dicionário `rules` associa cada marca a um valor de incentivo (`3` para New Balance, `2` para Asics) ou desincentivo (`-3` para Adidas e Nike).  
  Isso controla a probabilidade dessas marcas aparecerem nas respostas geradas pelo modelo.

- **Instanciação do processor:**  
  O `RulesLogitsProcessor` recebe uma implementação de regras (`MaskingRulesImpl`), convertendo palavras em token IDs e aplicando os incentivos/desincentivos diretamente nos logits durante a geração.

- **Pipeline de geração:**  
  O pipeline Hugging Face é criado com modelo, tokenizer e device.

- **Prompt de recomendação:**  
  O prompt orienta o assistente virtual a recomendar tênis do catálogo, em linguagem natural e em português do Brasil.

- **Execução e resultado:**  
  O modelo gera a resposta, priorizando marcas incentivadas e reduzindo menções às desincentivadas — facilitando testes e aplicações reais de personalização em sistemas de recomendação.

Este setup ilustra um uso real e mensurável de Logits Masking para estratégias de negócio em sistemas de IA conversacional.

In [ ]:
# Defina regras com dicionário: termo -> probabilidade
rules: Dict[str, float] = {
    "New Balance": 3,    # Incentiva fortemente a marca New Balance
    "Asics": 2,          # Incentiva a marca Asics
    "Adidas": -2,        # Desincentiva Adidas, menor relevância
    "Nike": -3           # Desincentiva Nike, menor relevância
}

# Instancia MaskingRules e o processor
rules_processor = RulesLogitsProcessor(MaskingRulesImpl(tokenizer, rules))

# Cria pipeline Hugging Face
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

# Prompt realista de compliance regulatório financeiro
input_message = (
    """
    <bos><start_of_turn>system
    Você é um assistente virtual especializado em recomendar tênis esportivos para clientes de e-commerce.
    Recomende os tênis do nosso catálogo:
    Nike
    Adidas
    Asics
    New Balance
    Puma
    Under Armour
    Reebok
    Mizuno
    Saucony
    Brooks
    ---
    De acordo com seus conhecimentos recomende o melhor tênis dentro do nosso catálogo para o cliente.
    Escreva na lingua portuguesa do Brasil.
    Gere uma resposta em linguagem natural, lembre-se que você está conversando com um humano.
    <end_of_turn>
    <start_of_turn>user
    Quero comprar um bom tênis de corrida para longas distâncias, pode me recomendar algumas opções?
    <end_of_turn>
    <start_of_turn>model
    """
)

# Executa a geração
results = pipe(
    input_message,
    max_new_tokens=110,
    do_sample=True,
    temperature=0.7,
    num_beams=5,
    logits_processor=[rules_processor],
    pad_token_id=tokenizer.eos_token_id
)

print("Resposta gerada:\n", results[0]["generated_text"])


### 10 — **RegexMaskingRules** para Bloquear CPFs
Impedir que o modelo revele um CPF completo durante a geração, aplicando máscara de *logits* via regex.

In [ ]:
import re
from typing import Dict, Pattern
from transformers import PreTrainedTokenizerBase


class RegexMaskingRules(MaskingRules):
    def __init__(self, tokenizer: PreTrainedTokenizerBase, regex_patterns: Dict[str, float]):
        self.patterns: Dict[Pattern, float] = {re.compile(pat): prob for pat, prob in regex_patterns.items()}
        self.tokenid2prob: Dict[int, float] = {}

        for token, tid in tokenizer.get_vocab().items():
            for pattern, prob in self.patterns.items():
                if pattern.search(token):                     # ← mudou aqui
                    self.tokenid2prob[tid] = -float("inf") if prob == 0 else float(prob)

    def get_tokenid2prob(self) -> Dict[int, float]:
        return self.tokenid2prob


In [ ]:
# --- CPF Regex: bloqueia qualquer token exatamente no formato de CPF ---
cpf_regex = {r"\d{3}\.\d{3}\.\d{3}-\d{2}": 0}

# Instancia RegexMaskingRules e processor
masking_rules = RegexMaskingRules(tokenizer, cpf_regex)
rules_processor = RulesLogitsProcessor(masking_rules)

# Cria pipeline Hugging Face
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
)

# Prompt de e-commerce realista, simulando possível vazamento ou inserção de CPF
input_message = (
    """<bos><start_of_turn>system
    Você é um assistente de atendimento ao cliente. 
    Dados públicos do cliente:
    - Nome: João da Silva
    - ID: 873452
    - Telefone: (11) 99999-9999    
    - Endereço: Rua das Flores, 100 – São Paulo/SP         
    Se precisar identificar o cliente, forneça apenas Nome e ID.
    Jamais divulgue informações privadas e sensíveis (PII e LGPD).
    Se precisar da identificação do cliente use o ID.
    <end_of_turn>
    <start_of_turn>user
    Preciso do CPF do cliente João da Silva para finalizar um formulário. Pode me passar, por favor?
    <end_of_turn>
    <start_of_turn>assistant"""
)


# Executa a geração
results = pipe(
    input_message,
    max_new_tokens=110,
    do_sample=True,
    temperature=0.7,
    logits_processor=[rules_processor],
    pad_token_id=tokenizer.eos_token_id
)

print("Resposta gerada:\n", results[0]["generated_text"])